# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case: a network with 5 nodes, without any structural contraint.
However, we introduce noise in polar coordinates and we try and use the power flow equations written with powers and our covariance matrix estimators in order to weight the noise.

In [132]:
import mlflow
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy import sparse
import seaborn as sns

In [133]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
import time
import sys
sys.path.insert(1, '..')

from src.simulation.noise import add_polar_noise_to_measurement
from src.models.matrix_operations import cross_multiply_measurements, make_measurements_matrix
from src.models.noise_transformation import power_covariance
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.models.regression import ComplexLasso
from src.identification.error_metrics import fro_error, error_metrics

/Users/jbrouill/PycharmProjects/eiv-grid-id/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:
mlflow.set_experiment('Toy problem with polar noise')

In [136]:
samples = 300
magnitude_sd = 0.01
phase_sd = 0.01
mlflow_params = {'samples': samples, 'magnitude_sd': magnitude_sd, 'phase_sd': phase_sd}

In [139]:
np.random.seed(11)

y_bus = np.array([
    [1+1j, 0, 0, 0, 0],
    [0, 2+1j, 0, 0, 0],
    [2+1j, 0, 1+1j, 0, 0],
    [0, 0, 0, 2+1j, 0],
    [0, 0, 0, 0, -1-1j],
])
nodes = y_bus.shape[0]

real_voltages = np.random.normal(1, 0.008, (samples, nodes)) + 1j*np.random.normal(0, 0.008, (samples, nodes))
real_currents = real_voltages @ y_bus

noisy_voltage = add_polar_noise_to_measurement(real_voltages, magnitude_sd, phase_sd)
noisy_current = add_polar_noise_to_measurement(real_currents, magnitude_sd, phase_sd)

real_powers = np.multiply(real_voltages,real_currents.conj())
powers = np.multiply(noisy_voltage,noisy_current.conj())

real_line_mat = cross_multiply_measurements(real_voltages)
line_mat = cross_multiply_measurements(noisy_voltage)

real_A_mat = make_measurements_matrix(real_voltages)
A_mat = make_measurements_matrix(noisy_voltage)

power_noise = powers - real_powers
line_mat_noise = line_mat - real_line_mat

b_vector = powers.reshape(powers.size)

# Standard TLS

In [138]:
mlflow.start_run(run_name='TLS')
eiv = TotalLeastSquares()
eiv.fit(A_mat, b_vector)
y_eiv = eiv.fitted_admittance_matrix.conj().reshape(5,5)

Exception: Run with UUID d56d91f7d4344b68ba7212a71d10117a is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True

In [ ]:
tls_metrics = error_metrics(y_bus, y_eiv)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(tls_metrics.__dict__)
mlflow.end_run()
tls_metrics

# Standard LASSO

In [ ]:
mlflow.start_run(run_name='Lasso')
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-5, 0, 50))
lasso.fit(A_mat, b_vector, True)
y_lasso = lasso.fitted_admittance_matrix.conj().T

In [ ]:
lasso_metrics = error_metrics(y_bus, y_lasso)
mlflow.log_param('lambda', lasso.best_trial.hyperparameters['lambda'])
mlflow.log_params(mlflow_params)
mlflow.log_metrics(lasso_metrics.__dict__)
mlflow.end_run()
lasso_metrics

In [ ]:
lasso.best_trial.hyperparameters

# L1-regularized TLS

In [ ]:
mlflow.start_run(run_name='S-TLS no cov')
tls_lambda = lasso.best_trial.hyperparameters['lambda']
max_iterations = 50
abs_tol = 10e-12
rel_tol = 10e-12
solver = cp.GUROBI
use_l1_penalty = True
use_cov_matrix = False
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
sparse_eiv.fit_with_vectored_data(line_mat, A_mat, b_vector)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix.conj()

In [ ]:
stls_metrics = error_metrics(y_bus, y_sparse_eiv)
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_param('use_cov_matrix', use_cov_matrix)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_metrics.__dict__)
y_errors = pd.Series([fro_error(y_bus.conj(), i.fitted_parameters) for i in sparse_eiv.iterations])
targets = pd.Series([i.target_function for i in sparse_eiv.iterations])
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()
stls_metrics

In [ ]:
y_errors.plot(title='Fro error on Y');

In [ ]:
targets.plot(title='Target function');

# L1-regularized TLS with estimated covariance matrix

In [ ]:
sigma_power = power_covariance(noisy_voltage.reshape(noisy_voltage.size), noisy_current.reshape(noisy_current.size),
                               magnitude_sd, phase_sd, magnitude_sd, phase_sd)
voltages1 = noisy_voltage.repeat(noisy_voltage.shape[1],1).flatten()
voltages2 = np.tile(noisy_voltage, (1, noisy_voltage.shape[1])).flatten()
sigma_line_mat = power_covariance(voltages1, voltages2, magnitude_sd, phase_sd, magnitude_sd, phase_sd)

inv_sigma_power = sparse.linalg.inv(sigma_power)
inv_sigma_line_mat = sparse.linalg.inv(sigma_line_mat)

In [ ]:
mlflow.start_run(run_name='S-TLS')
use_cov_matrix = True
sparse_eiv_cov = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
sparse_eiv_cov.fit_with_vectored_data(line_mat, A_mat, b_vector, inv_sigma_line_mat, inv_sigma_power)
y_sparse_eiv_cov = sparse_eiv_cov.fitted_admittance_matrix.conj()

In [ ]:
stls_cov_metrics = error_metrics(y_bus, y_sparse_eiv_cov)
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_param('use_cov_matrix', use_cov_matrix)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_cov_metrics.__dict__)
y_errors = pd.Series([fro_error(y_bus.conj(), i.fitted_parameters) for i in sparse_eiv_cov.iterations])
targets = pd.Series([i.target_function for i in sparse_eiv_cov.iterations])
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()
stls_cov_metrics

In [ ]:
y_errors.plot(title='Fro error on Y');

In [ ]:
targets.plot(title='Target function');

# L1-regularized TLS with covariance matrix based on actual measurements

In [ ]:
sigma_power = power_covariance(real_voltages.reshape(real_voltages.size), real_currents.reshape(real_currents.size),
                               magnitude_sd, phase_sd, magnitude_sd, phase_sd)
voltages1 = real_voltages.repeat(real_voltages.shape[1],1).flatten()
voltages2 = np.tile(real_voltages, (1, real_voltages.shape[1])).flatten()
sigma_line_mat = power_covariance(voltages1, voltages2, magnitude_sd, phase_sd, magnitude_sd, phase_sd)

inv_sigma_power = sparse.linalg.inv(sigma_power)
inv_sigma_line_mat = sparse.linalg.inv(sigma_line_mat)

In [ ]:
mlflow.start_run(run_name='S-TLS')
use_cov_matrix = True
sparse_eiv_cov = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
sparse_eiv_cov.fit_with_vectored_data(line_mat, A_mat, b_vector, inv_sigma_line_mat, inv_sigma_power)
y_sparse_eiv_cov = sparse_eiv_cov.fitted_admittance_matrix.conj()

In [ ]:
stls_cov_metrics = error_metrics(y_bus, y_sparse_eiv_cov)
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_param('use_cov_matrix', use_cov_matrix)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_cov_metrics.__dict__)
y_errors = pd.Series([fro_error(y_bus.conj(), i.fitted_parameters) for i in sparse_eiv_cov.iterations])
targets = pd.Series([i.target_function for i in sparse_eiv_cov.iterations])
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()
stls_cov_metrics

In [ ]:
y_errors.plot(title='Fro error on Y');

In [ ]:
targets.plot(title='Target function');

In [ ]:
sns.heatmap(np.abs(y_bus));

In [ ]:
sns.heatmap(np.abs(y_sparse_eiv_cov.toarray()));